In [6]:
# Carga de elementos a utilizar globalmente
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

melbourne_file_path = 'assets/input/melb_data.csv'
melb_home_data = pd.read_csv(melbourne_file_path)
filtered_melb_data = melb_home_data.dropna(axis=0)

y = filtered_melb_data.Price
features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = filtered_melb_data[features]

melb_model = DecisionTreeRegressor(random_state=1)
melb_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

# Validación del modelo
Es la medida de **calidad de nuestro modelo** y conocer este valor nos permitirá mejorarlo iterativamente.
El valor más importante es, en general, la **precisión de la predicción**.

Un error muy común es calcular esta precisión basándose en predicciones y valor real del mismo set de entreamiento.
La métrica a utilizar es el **mean absolute error (MAE)**.

*NOTA:* una primera inquietud que puede surgir aquí es cómo puede calcular el error si para ello necesitaría saber el valor real del *target*, que es justamente el que estoy queriendo predecir. La respuesta a ello es que se utilizará un **set de datos específico para validación**, el cual incluirá el valor conocido del *target* que nos permitirá saber la dispersión que tenga nuestra predicción.

In [9]:
from sklearn.metrics import mean_absolute_error

# Obtengo Predicciones y calculo su mean_error
# OJO: esto es ilustrativo, se realizaron las predicciones 
# sobre el mismo set de entrenamiento, lo cual esta mal
predictions = melb_model.predict(X)
mae = mean_absolute_error(y, predictions)

print('Error medio: {}'.format(mae))

Error medio: 1115.7467183128902


*DUDA:* se dijo que está mal calcular el error sobre el mismo set de entreamiento, y eso lo entiendo, pero no debería dar 0 el error? o es que el proceso de fit igualmente tiene cierto error (mínimo en este caso, ya que el valor del target predicho será practicamente igual)

## In-sample error
El error a la hora de calcular la precisión de la manera en que se demostró antes es lo que se conoce como **in-sample error**, donde se utiliza el mismo set de datos tanto para la construcción del modelo como para su validación.

Una forma de entender conceptualmente este error: supongamos que se tiene un set de datos de determinadas casas, en las cuales una característica como ser "color de la puerta: verde" hace que la casa tenga un precio alto. Nuestro modelo, al buscar patrones, detectará dicho comportamiento y lo replicará en las predicciones, es decir, al ver una casa con puerta verde, predecirá un precio alto. Si esto lo validamos dentro del mismo set de datos, arrojará una gran exactitud, pero ahora, supongamos que esa correlación no se da en otra ciudad, de esta manera, al aplicar nuestro modelo en este otro set de datos, tendremos un gran error.

**Para evitar esto, la medición de la precisión de nuestro modelo se realizá sobre un nuevo set de datos**, distinto al que se utilizó para el entrenamiento, denominado **validation data**.
La manera más sencilla de hacerlo es excluir cierta cantidad de datos (de nuestro set de entreamiento) del proceso de creación del modelo y luego utilizarlo para medir la precisión con datos que nunca haya visto.

In [10]:
from sklearn.model_selection import train_test_split

# Separo set de datos original
# Uno para fit y otro para validar
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

melb_model.fit(train_X, train_y)
predictions = melb_model.predict(val_X)

mae = mean_absolute_error(val_y, predictions)
print('Error medio: {}'.format(mae))

Error medio: 273518.01872175594


Como se puede observar, el error con este modelo es de aproximadamente 275000 (siendo el valor promedio de una casa 1M, es decir, un 25% de error, muy alto) mientras que como lo habíamos calculado anteriormente 'in-sample' nos daba aproximadamente 1000, la cual era obviamente una medida equivocada. Esta es la importancia de realizar correctamente la validación de nuestro modelo ya que podemos pasar de un modelo que *equivocadamente* nos parece excelente prediciendo, a uno que es prácticamente inútil.